In [3]:
import pandas as pd
from amplpy import AMPL, Environment

In [4]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [5]:
ampl.reset()

In [6]:
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))

minos
cplex


In [7]:
model="""
#PART1: DECISION VARIABLES/UNKNOWN VARIABLES
var x1 >= 0;
var x2 >= 0;
var x3 >= 0;

#PART2: OBJECTIVE FUNCTION
maximize objfunc: x1 + x2 + 2*x3;

#PART3: CONSTRAINTS
s.t. c1: x2 + 2*x3 <= 3;
s.t. c2: -x1 + 3*x3 <= 2;
s.t. c3: 2*x1 + x2 + x3 <= 1;

#Each constraint has a name. That's how we will refer to each constraint.
#The objective function has a name.
#Notice c3 is an equality constraint
"""

In [8]:
ampl.eval(model)

In [9]:
print(ampl.getObjective('objfunc').get())
ampl.eval('expand objfunc;')

maximize objfunc:
	x1 + x2 + 2*x3;
maximize objfunc:
	x1 + x2 + 2*x3;



In [10]:
for i in ampl.getConstraints():
    print(i[1])
#print(ampl.getConstraint('c1'))
#print(ampl.getConstraint('c2'))
#print(ampl.getConstraint('c3'))

subject to c1: x2 + 2*x3 <= 3;
subject to c2: -x1 + 3*x3 <= 2;
subject to c3: 2*x1 + x2 + x3 <= 1;


In [11]:
ampl.eval('expand c1;')
ampl.eval('expand c2;')
ampl.eval('expand c3;')

subject to c1:
	x2 + 2*x3 <= 3;

subject to c2:
	-x1 + 3*x3 <= 2;

subject to c3:
	2*x1 + x2 + x3 <= 1;



In [12]:
ampl.eval('expand x1;')
ampl.eval('expand x2;')

Coefficients of x1:
	c2       -1
	c3        2
	objfunc   1

Coefficients of x2:
	c1       1
	c3       1
	objfunc  1



In [13]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 3



In [14]:
ampl.solve()

CPLEX 12.8.0.0: optimal solution; objective 1.666666667
2 dual simplex iterations (1 in phase I)


In [15]:
print(ampl.getObjective('objfunc').get().value())

1.6666666666666665


#### Shadow Price or Dual Price or Marginal Price

In [16]:
for i in ampl.getConstraints():
    print(i[1].getValues().toPandas())

   c1.dual
0      0.0
    c2.dual
0  0.333333
   c3.dual
0      1.0


In [17]:
for i in ampl.getVariables():
    print(i[1].getValues())

   x1.val   
     0      

   x2.val   
0.33333333333333337

   x3.val   
0.66666666666666663



### Decision Variable Solution and Reduced Cost

In [18]:
X_result = list()
for v in ampl.getVariables():
    X_result.append([
        v[0],
        v[1].lb(), 
        v[1].value(),
        v[1].ub(),
        v[1].rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
x1,0.0,0.000000,inf,-0.666667
x2,0.0,0.333333,inf,0.000000
x3,0.0,0.666667,inf,0.000000
